In [1]:
import os
import sys
import numpy as np

from astropy.io import fits

import photutils
from photutils import EllipticalAperture

import matplotlib.pyplot as plt
%matplotlib inline

# use nicer set of plot parameters (from astropy www)
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)

In [2]:
def computeSkyStats(f):
    ### This procedure reads a background file, compiles extant statistics, and computes systematic errors 
    ### in assumed sky level
    
    lines = f.readlines()
    f.close()
    
    index = [j + 1 for j, line in enumerate(lines) if line.strip()[-3:] == 'std']
    #sky_nPix = float(lines[index[0]].strip().split()[0])
    #sky_stdDev = float(lines[index[0]].strip().split()[-1])
    #sky_stdErr = sky_stdDev / np.sqrt(sky_nPix)
    skyStats = np.array(lines[index[0]].strip().split(), dtype = float)
    
    index = [j + 1 for j, line in enumerate(lines) if 'CofG'.lower() in line.lower()]
    try:
        sky = float(lines[index[-1]].strip())
    except:
        sky = float(lines[index[-1] + 1].strip())
        
    skyStats = np.hstack([skyStats, sky])
    sky_stdDev = np.std(np.hstack([skyStats[1], skyStats[2], skyStats[3], skyStats[-1]]), ddof = 1)
    
    #sky_stdErr = sky_stdDev
    sky_stdErr = sky_stdDev / 2.0
    #sky_stdErr = skyStats[4] / np.sqrt(skyStats[0])
    
    return skyStats, sky_stdDev, sky_stdErr

In [3]:
def checkHeaderCards(path, galaxyId, band):
    ### This procedure inspects the header of an image stack for HISTORY cards that record the sky values 
    ### subtracted from individual images comprising the stack and prints them to screen
    
    try:
        f = path + galaxyId + band + '.fits'
        hduList = fits.open(f)
    except:
        f = path + galaxyId + band + '_ls.fits'
        hduList = fits.open(f)
        
    if ('HISTORY' in hduList[0].header):
        history = fits.getval(f, 'HISTORY', ext = 0)
    else:
        history =[]
        
    #print ('N_history cards:', len(history))
    for j in range(len(history)):
        index = history[j].find('Sky=')
        # skip HISTORY cards not housing sky values
        if index < 0: continue
        #print (band, float(history[j][index + 4:]))

In [4]:
def getTotalMagAndRad(path, galaxyId, band):
    ### This procedure searches the results file to obtain the total magnitude and effective radius for a galaxy
    
    try:
        f = open(path + galaxyId + band + 'fit.results4', 'r')
    except:
        f = open(path + galaxyId + band + 'ls_fit.results4', 'r')
        
    lines = f.readlines()
    f.close()
    
    index = [j + 5 for j, line in enumerate(lines) if line.strip()[:4] == 'NGVS']
    mag = float(lines[index[0]].strip().split()[2])
    # express radius in units of pixels
    re = float(lines[index[0]].strip().split()[1]) / 0.1858
    
    return mag, re

In [5]:
def getIsophotalParms(path, galaxyId, band):
    ### This procedure searches the growth curve file to obtain the center, semi-major axis, ellipticity, 
    ### position angle and number of enclosed pixels of the isophote containing the total light for a galaxy
    
    try:
        f = open(path + galaxyId + band + 'cg.dat.cg4', 'r')
    except:
        f = open(path + galaxyId + band + 'ls_cg.dat.cg4', 'r')
        
    lines = f.readlines()
    f.close()
    
    index = [j for j, line in enumerate(lines) if (float(line.strip().split()[1]) - mag) <= 0.001]
    
    rad = [float(lines[index[0] - 1].strip().split()[2]), float(lines[index[0]].strip().split()[2])]
    ellip = [float(lines[index[0] - 1].strip().split()[5]), float(lines[index[0]].strip().split()[5])]
    pa = [float(lines[index[0] - 1].strip().split()[7]), float(lines[index[0]].strip().split()[7])]
    x0 = [float(lines[index[0] - 1].strip().split()[9]), float(lines[index[0]].strip().split()[9])]
    y0 = [float(lines[index[0] - 1].strip().split()[11]), float(lines[index[0]].strip().split()[11])]
    nPix = [float(lines[index[0] - 1].strip().split()[15]), float(lines[index[0]].strip().split()[15])]
    
    return rad, ellip, pa, x0, y0, nPix

In [6]:
def computeRandomErr(path, galaxyId, band, mag, rad, ellip, pa, x0, y0):
    ### This procedure uses an image of the random error per pixel to compute the error in the total magnitude 
    ### for a galaxy due to the source + background photons.
    
    f = galaxyId + band + 'sig.fits'
    if f not in os.listdir(baseDirs[2]):
        f = galaxyId + band + 'ls_sig.fits'
        
    f = baseDirs[2] + f
    sigmaImg = fits.getdata(f, ext = 0) ** 2
    
    positions = [(x0[0], y0[0])]
    apertures = EllipticalAperture(positions, a = rad[0], b = (1. - ellip[0]) * rad[0], theta = pa[0])
    sigmaSky_rand = (2.5 / np.log(10.)) * np.sqrt(apertures.do_photometry(sigmaImg)[0][0]) / \
        (10.0 ** (-0.4 * (mag - 30.0)))
    
    #plt.figure(figsize = [20, 20])
    #plt.imshow(sigmaImg, cmap = 'gray', vmin = 0.42286697, vmax = 0.5, origin = 'lower')
    #plt.colorbar()
    #apertures.plot(origin = (0, 0))
    
    return sigmaSky_rand

In [7]:
# ...
# ACCESS SKY STATISTICS AND CALCULATE THE ASSOCIATED 1-SIGMA ERRORS ON GALAXY MAGNITUDES

calErr = [0.0283, 0.0141, 0.0141, 0.0141, 0.0224]

baseDirs = ['../../massMaps/PP/tar-balls/FITS_FREE/', 
            '../../massMaps/PP/tar-balls/FIXISO/', 
            './Volumes/Disk2/NGVS/Profiles/FAST_RESULTS/FINAL_ISOPHOTES/PP/FITS_FREE/', 
            './Volumes/Disk2/NGVS/Profiles/FAST_RESULTS/FINAL_ISOPHOTES/PP/FIXISO/']

# list contents of free and fixed isophotal analysis archives
dirs = [os.listdir(baseDirs[0]), os.listdir(baseDirs[1])]

# remove archives from previous run (if still present)
try:
    os.system('rm -rv ./Volumes/*')
    os.rmdir('./Volumes/')
except:
    print ('No directory to remove\n')
    
# open ASCII file to hold results
f1 = open('skyErrorEstimates.txt', 'w')
f1.write('#                        id   b      mag     nPix_g     nPix_s       sky    skyErr_s   skyStdErr_s' + \
         '    magErr_r    magErr_s    magErr_t    delMag_p    delMag_m\n')

for i in range(1, len(dirs[0])):
    # extract tar archives for free and fixed isophotal analyses
    os.system('tar -xvzf ' + baseDirs[0] + dirs[0][i])
    os.system('tar -xvzf ' + baseDirs[1] + dirs[1][i])
    
    # generate base of filenames stored in archives
    if os.listdir(baseDirs[2])[0][7] == '.':
        if 'VCC' in os.listdir(baseDirs[2])[0]:
            index = os.listdir(baseDirs[2])[0].find('_')
            #index = os.listdir(baseDirs[2])[0].find('_', index + 1)
            index = os.listdir(baseDirs[2])[0].find('.', index + 1)
            galaxyId = os.listdir(baseDirs[2])[0][:index]
        else:
            galaxyId = os.listdir(baseDirs[2])[0][:27]
    else:
        galaxyId = os.listdir(baseDirs[2])[0][:32]
        
    
    # cycle through each band, determining sky subtraction errors in each
    print (i, ' ', dirs[0][i][:-7], ' ', galaxyId)
    for j, band in enumerate(['_U_', '_G_', '_R_', '_I_', '_Z_']):
        # check for existence of background file; if absent, skip the current band
        strings = [galaxyId + band + 'back.dat', 
                   galaxyId + band + 'ls_back.dat']
        if strings[0] in os.listdir(baseDirs[2]):
            f = open(baseDirs[2] + galaxyId + band + 'back.dat', 'r')
        elif strings[1] in os.listdir(baseDirs[2]):
            f = open(baseDirs[2] + galaxyId + band + 'ls_back.dat', 'r')
        else:
            continue
            
        # obtain statistics related to residual sky level in image
        skyStats, sky_stdDev, sky_stdErr = computeSkyStats(f)
        
        #-----
        
        # print sky values subtracted from images comprising stack containing galaxy
        #checkHeaderCards(baseDirs[2], galaxyId, band[:-1])
        
        #-----
        
        # print value of OBJECT card (should identify set of stacks in which galaxy center located)
        #print (fits.getval(filename, 'OBJECT', ext = 0))
        
        #-----
        
        # obtain total magnitude and effective radius for galaxy
        mag, re = getTotalMagAndRad(baseDirs[3], galaxyId, band)
        
        #-----
        
        # obtain parameters of isophote along CoG containing total light for galaxy
        rad, ellip, pa, x0, y0, nPix = getIsophotalParms(baseDirs[3], galaxyId, band)
        
        #-----
        
        # compute random and systematic contributions of sky subtraction to error budget
        magErr_r = computeRandomErr(baseDirs[2], galaxyId, band, mag, rad, ellip, pa, x0, y0)
        magErr_s = (2.5 / np.log(10.)) * (np.sqrt(nPix[0]) * sky_stdDev) / (10.0 ** (-0.4 * (mag - 30.0)))
        magErr_t = (2.5 / np.log(10.)) * np.sqrt(10.0 ** (-0.4 * (mag - 30.0)) + (nPix[0] * skyStats[-1]) + 
                                                 (nPix[0] * sky_stdDev) ** 2 + 
                                                 (calErr[j] * (10.0 ** (-0.4 * (mag - 30.0))) / 
                                                  (2.5 / np.log(10.))) ** 2) / (10.0 ** (-0.4 * (mag - 30.0)))
        
        #-----
        
        # compute differences in total magnitude if sky level changed by +/-1 sigma of systematic error
        delMag = [-2.5 * np.log10(10.0 ** (-0.4 * (mag - 30.0)) + (nPix[0] * (sky_stdErr))) + 30.0, 
                  -2.5 * np.log10(10.0 ** (-0.4 * (mag - 30.0)) - (nPix[0] * (sky_stdErr))) + 30.0]
        delMag[0] = mag - delMag[0]
        delMag[1] = delMag[1] - mag
        
        # write results to file
        f1.write('%s %3s %8.3f %10d %10d %9.3f %11.3e %13.3e %11.3e %11.3e %11.3e %11.3e %11.3e\n' % 
                 (dirs[0][i][:-7], band[1], mag, nPix[0], skyStats[0], skyStats[-1], sky_stdDev, sky_stdErr, \
                  magErr_r, magErr_s, magErr_t, delMag[0], delMag[1]))
        
    os.system('rm -rv ./Volumes/*')
    os.rmdir('./Volumes/')
    
f1.close()

1   NGVSJ12:26:20.07+12:30:37.1   NGVS186.58353+12.510362-1+1
2   NGVSJ12:26:20.39+12:34:27.3   NGVS12:26:20.194+12:34:27.32-1+1
3   NGVSJ12:26:22.61+12:47:11.0   NGVS12:26:22.589+12:47:10.51-1+1
4   NGVSJ12:26:23.64+13:22:24.7   NGVS12:26:23.604+13:22:23.65-1+1
5   NGVSJ12:26:24.04+12:25:00.5   NGVS12:26:24.034+12:25:00.20-1+0
6   NGVSJ12:26:26.21+12:39:10.6   NGVS12:26:26.203+12:39:10.83-1+1
7   NGVSJ12:26:26.30+11:44:08.0   NGVS12:26:26.321+11:44:07.82-1+0
8   NGVSJ12:26:26.97+12:54:23.6   NGVS186.61202+12.906951-1+1


/Users/roedigerj/anaconda/envs/astro/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in log10


9   NGVSJ12:26:27.83+12:45:52.7   NGVS12:26:27.830+12:45:52.86-1+1
10   NGVSJ12:26:28.06+12:55:14.2   NGVS12:26:28.075+12:55:13.63-1+1
11   NGVSJ12:26:31.31+12:29:32.4   NGVS12:26:31.308+12:29:32.58-1+1
12   NGVSJ12:26:32.25+12:36:38.5   NGVS12:26:32.218+12:36:37.88-1+1
13   NGVSJ12:26:32.68+13:25:25.8   NGVS12:26:32.669+13:25:25.29-1+1
14   NGVSJ12:26:33.21+12:44:34.7   NGVS12:26:33.214+12:44:35.65-1+1
15   NGVSJ12:26:35.84+13:22:44.7   NGVS12:26:35.796+13:22:44.10-1+1
16   NGVSJ12:26:36.32+12:48:10.0   NGVS12:26:36.336+12:48:09.36-1+1
17   NGVSJ12:26:37.74+12:43:48.1   NGVS12:26:37.728+12:43:48.22-1+1
18   NGVSJ12:26:38.09+11:53:30.7   NGVS12:26:38.076+11:53:30.53-1+0
19   NGVSJ12:26:38.25+13:04:44.2   NGVS12:26:38.232+13:04:43.79-1+1
20   NGVSJ12:26:39.81+12:30:48.8   NGVS12:26:39.804+12:30:48.37-1+1
21   NGVSJ12:26:41.15+12:50:43.5   NGVS12:26:41.165+12:50:43.00-1+1
22   NGVSJ12:26:42.11+13:22:33.3   NGVS12:26:42.120+13:22:32.51-1+1
23   NGVSJ12:26:43.31+12:17:44.0   NGVS12:26:43.3

130   NGVSJ12:28:32.13+12:32:09.7   NGVS12:28:32.119+12:32:09.20-1+1
131   NGVSJ12:28:32.40+11:44:40.7   NGVS12:28:32.477+11:44:38.14-1+0
132   NGVSJ12:28:35.75+12:10:57.2   NGVS12:28:35.750+12:10:57.06-1+0
133   NGVSJ12:28:36.07+11:40:16.5   NGVS12:28:36.060+11:40:16.60-1+0
134   NGVSJ12:28:37.88+12:51:42.0   NGVS12:28:37.200+12:51:37.00-1+1
135   NGVSJ12:28:39.87+12:58:40.5   NGVS12:28:39.835+12:58:40.33-1+1
136   NGVSJ12:28:41.71+12:54:57.2   NGVS187.17382+12.915885_VCC1122
137   NGVSJ12:28:42.66+12:32:59.4   NGVS12:28:42.670+12:32:59.25-1+1
138   NGVSJ12:28:43.31+11:45:18.1   NGVS187.1805+11.755034_VCC1125
139   NGVSJ12:28:44.65+11:59:37.2   NGVS12:28:44.614+11:59:36.60-1+0
140   NGVSJ12:28:44.91+12:48:34.3   NGVS12:28:44.894+12:48:33.35-1+1
141   NGVSJ12:28:45.79+12:01:18.6   NGVS12:28:45.758+12:01:17.66-1+0
142   NGVSJ12:28:46.92+12:38:31.5   NGVS12:28:46.930+12:38:30.87-1+1
143   NGVSJ12:28:47.37+12:49:48.5   NGVS12:28:47.362+12:49:47.60-1+1
144   NGVSJ12:28:48.93+11:53:10.4   N

250   NGVSJ12:30:32.18+12:51:51.2   NGVS12:30:32.194+12:51:50.40+0+1
251   NGVSJ12:30:33.32+12:54:02.3   NGVS12:30:33.326+12:54:01.05+0+1
252   NGVSJ12:30:34.65+12:27:29.2   NGVS12:30:34.639+12:27:29.44+0+1
253   NGVSJ12:30:35.12+13:11:20.2   NGVS12:30:35.122+13:11:19.86+0+1
254   NGVSJ12:30:37.24+12:46:09.2   NGVS12:30:37.234+12:46:08.45+0+1
255   NGVSJ12:30:37.35+13:00:33.3   NGVS12:30:37.330+13:00:32.90+0+1
256   NGVSJ12:30:40.41+12:37:17.8   NGVS12:30:40.414+12:37:17.30+0+1
257   NGVSJ12:30:42.65+12:47:26.1   NGVS12:30:42.655+12:47:25.97+0+1
258   NGVSJ12:30:46.32+12:05:56.7   NGVS12:30:46.332+12:05:56.62+0+0
259   NGVSJ12:30:46.32+12:36:49.5   NGVS12:30:46.308+12:36:49.16+0+1
260   NGVSJ12:30:46.88+13:12:50.4   NGVS12:30:46.949+13:12:49.85+0+1
261   NGVSJ12:30:47.20+11:32:15.4   NGVS12:30:47.146+11:32:15.23+0+0
262   NGVSJ12:30:48.58+12:02:42.7   NGVS12:30:48.538+12:02:42.22+0+0
263   NGVSJ12:30:49.03+13:13:25.8   NGVS12:30:49.085+13:13:24.28+0+1
264   NGVSJ12:30:49.42+12:23:28.0 

369   NGVSJ12:33:11.87+12:42:55.7   NGVS12:33:11.866+12:42:55.93+0+1
370   NGVSJ12:33:14.01+12:51:28.2   NGVS12:33:13.994+12:51:27.80+0+1
371   NGVSJ12:33:14.02+11:46:53.6   NGVS12:33:14.004+11:46:53.42+0+0
372   NGVSJ12:33:15.73+11:52:07.0   NGVS12:33:15.679+11:52:06.52+0+0
373   NGVSJ12:33:15.83+13:13:10.3   NGVS12:33:15.821+13:13:10.35+0+1
374   NGVSJ12:33:16.88+12:16:56.2   NGVS12:33:16.922+12:16:56.08+0+0
375   NGVSJ12:33:16.91+12:34:54.5   NGVS12:33:16.860+12:34:54.57+0+1
376   NGVSJ12:33:17.19+11:37:36.4   NGVS12:33:17.208+11:37:36.83+0+0
377   NGVSJ12:33:17.38+12:34:54.5   NGVS12:33:17.275+12:34:53.94+0+1
378   NGVSJ12:33:19.79+12:51:12.5   NGVS12:33:19.800+12:51:12.61+0+1
379   NGVSJ12:33:22.53+11:38:29.4   NGVS12:33:22.567+11:38:28.95+0+0
380   NGVSJ12:33:24.73+12:24:11.3   NGVS12:33:24.715+12:24:10.89+0+0
381   NGVSJ12:33:25.21+13:24:58.5   NGVS12:33:25.291+13:25:00.25+0+1
382   NGVSJ12:33:29.44+13:17:22.8   NGVS12:33:29.446+13:17:22.81+0+1
383   NGVSJ12:33:30.72+13:00:21.5 

In [ ]:
# PLAN

In [ ]:
# ISSUES
#  lack sufficient info to directly estimate errors for every galaxy in survey
#  applying trend cast wrt CoG magnitudes to Galfit magnitudes

In [8]:
# READ MAGNITUDES AND TOTAL ERRORS FOR GALAXIES HAVING DIRECT SKY ERROR ESTIMATES

#f = open('skyErrorEstimates_stdDev.txt', 'r')
f = open('skyErrorEstimates_stdDev_wSkyInPoissonTermAndCalErr.txt', 'r')
lines = f.readlines()
f.close()

bands = np.array([line.strip().split()[1] for line in lines[1:]])
mags = np.array([float(line.strip().split()[2]) for line in lines[1:]])
errs = np.array([float(line.strip().split()[10]) for line in lines[1:]])

In [10]:
# READ (CoG) MAGNITUDES FOR EVERY GALAXY FROM FULL SURVEY

f = open('mags_ngvsFull_err=deltaMag.txt', 'r')
lines = f.readlines()
f.close()

ngvsID = np.array([line.strip().split()[0] for line in lines])

mags_cog_full = np.zeros([len(lines), 5])
mags_cog_full[:, 0] = [float(line.strip().split()[3]) for line in lines]
mags_cog_full[:, 1] = [float(line.strip().split()[1]) for line in lines]
mags_cog_full[:, 2] = [float(line.strip().split()[5]) for line in lines]
mags_cog_full[:, 3] = [float(line.strip().split()[7]) for line in lines]
mags_cog_full[:, 4] = [float(line.strip().split()[9]) for line in lines]

In [11]:
# ...

import statsmodels.api as sm

from scipy.interpolate import interp1d

minErr = [0.0283, 0.0141, 0.0141, 0.0141, 0.0224]

magErrs_cog_pred = -100.0 * np.ones(mags_cog_full.shape)

for j, band in enumerate(['U', 'G', 'R', 'I', 'Z']):
    
    # grab mags and errors for band, and stash in single array (sorted by mag)
    selection = (bands == band)
    data = np.stack([mags[selection], errs[selection]], axis = 1)
    data = data[data[:, 0].argsort()]
    
    # replace errors lying below value for calibration error in band
    mask = (data[:, 1] < minErr[j])
    #data[mask, 1] = minErr[j]
    
    # determine LOWESS trend between selected mags and errors (write to file, omitting repeated values)
    lowess = sm.nonparametric.lowess(data[:, 1], data[:, 0], frac = 0.3, it = 3)
    f = open('lowess_totErrsVsMags_' + band + '_x=cg.txt', 'w')
    
    x, indxs = np.unique(lowess[:, 0], return_index = True)
    y = lowess[indxs, 1]
    for i in range(lowess.shape[0]):
        if i in indxs:
            f.write('%8.4f %9.5f\n' % (lowess[i, 0], lowess[i, 1]))
    f.close()
    
    # interpolate LOWESS solution and predict magnitude errors therefrom
    mask = (mags_cog_full[:, j] > 0.)
    func = interp1d(x, y, kind = 'cubic', bounds_error = False, fill_value = "extrapolate")
    magErrs_cog_pred[mask, j] = func(mags_cog_full[mask, j])
    
    # fix erroneous values caused by extrapolation
    mask = ((mags_cog_full[:, j] < lowess[0, 0]) & (mags_cog_full[:, j] > 0.))
    magErrs_cog_pred[mask, j] = lowess[0, 1]
    
    mask = (mags_cog_full[:, j] > lowess[-1, 0])
    magErrs_cog_pred[mask, j] = lowess[-1, 1]
    
    # add scatter to predicted errors
    count = 0
    for i in range(mags_cog_full.shape[0]):
        if mags_cog_full[i, j] < 0.:
            continue
            
        # find 10 closest neighours from pilot field to galaxy in mag-space 
        idx = np.searchsorted(data[:, 0], mags_cog_full[i, j])
        argMin = np.max([idx - 5, 0])
        argMax = np.min([argMin + 10, data.shape[0]])
        
        # find number of neighbours from pilot field lying within X mag of galaxy
        mask = (np.abs(data[:, 0] - mags_cog_full[i, j]) <= 0.5)
        
        # use fixed or adaptive bin to compute standard deviation
        if np.sum(mask) < 10:
            #scratch = data[argMin:argMax, 1] - lowess[argMin:argMax, 1]
            scratch = np.log10(data[argMin:argMax, 1]) - np.log10(lowess[argMin:argMax, 1])
            count += 1
        else:
            #scratch = data[mask, 1] - lowess[mask, 1]
            scratch = np.log10(data[mask, 1]) - np.log10(lowess[mask, 1])
            
        delta = np.random.normal(loc = np.log10(magErrs_cog_pred[i, j]), scale = 
                                 np.std(scratch, ddof = 1, dtype = np.float64))
        #magErrs_cog_pred[i, j] = delta
        magErrs_cog_pred[i, j] = 10.0 ** delta
        
    #print(count)
        
    # replace errors lying below value for calibration error in band
    mask = ((magErrs_cog_pred[:, j] < minErr[j]) & (mags_cog_full[:, j] > 0.))
    magErrs_cog_pred[mask, j] = minErr[j]
    
    mask = ((magErrs_cog_pred[:, j] < 10.0 ** (0.2 * (mags_cog_full[:, j] - 30.) + 0.0357)) & \
            (mags_cog_full[:, j] > 0.))
    magErrs_cog_pred[mask, j] = 10.0 ** (0.2 * (mags_cog_full[mask, j] - 30.) + 0.0357)
    #break

/Users/roedigerj/anaconda/envs/astro/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [12]:
# ...

f = open('mags_ngvsFull_err=sky+Gxy.txt', 'w')
for i in range(mags_cog_full.shape[0]):
    string = '%s' % (ngvsID[i])
    for j in range(0, 5):
        if j < 5 or j > 9:
            string += '%12.4f %12.5f' % (mags_cog_full[i, j], magErrs_cog_pred[i, j])
            
    f.write(string + '\n')
f.close()

In [ ]:
# DETERMINE, PLOT, AND STORE LOWESS FITS FOR TRENDS BTWN TOTAL ERRORS AND COG MAGNITUDES

idxs = [1, 0, 2, 3, 4]
bands = ['u', 'g', 'r', 'i', 'z']

colors = ['purple', 'blue', 'green', 'orange', 'red']
xlims = [[8.1, 24.9], [9.5, 27.0], [7.6, 25.9], [7.0, 25.9], [6.5, 25.9]]
ylims = [[-0.019, 0.199], [-0.099, 1.199], [-0.019, 0.199], [-0.049, 0.549], [-0.049, 0.999]]

# cycle through each band
for idx, band, color, xlim, ylim in zip(idxs, bands, colors, xlims, ylims):
    
    # plot data and LOWESS solution
    fig = plt.figure(figsize = (20, 10))
    ax1 = plt.subplot(111)
    ax1.scatter(mags_PP[:, idx], magErrs_PP[:, idx], facecolor = color, alpha = 0.5)
    ax1.plot(lowess[:, 0], lowess[:, 1], c = 'k', lw = 3)
    ax1.set_xlim(xlim)
    ax1.set_ylim(ylim)
    plt.savefig('magErrsVsMags_' + band + '_x=cg.pdf');
    